In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [70]:
import nltk
import re
import csv

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

import bq_helper
from bq_helper import BigQueryHelper

import warnings
warnings.filterwarnings('ignore', message=r'Label not .* is present in all training examples.')

pd.set_option('display.max_colwidth', 300)

In [71]:
from google.cloud import bigquery
import tensorflow as tf
import string
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import f1_score, accuracy_score

**Accessing Big Query data**

In [73]:
stack_overflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                         dataset_name="stackoverflow")

Using Kaggle's public dataset BigQuery integration.


In [74]:
bg = BigQueryHelper("bigquery-public-data", "stackoverflow")
bg.list_tables()

Using Kaggle's public dataset BigQuery integration.


['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [75]:
bg.head("stackoverflow_posts", num_rows=1)


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,376158,None,<p>Assuming // do something is more than just the comment take out the </p> <p>// do something </p> <p>code and see if it still crashes. Add stuff back one at a time till it crashes again</p>,None,None,0,None,2008-12-17 21:52:23.097000+00:00,None,2008-12-17 21:52:23.097000+00:00,None,None,None,n8wrl,37710,361115,2,0,,None


In [76]:
bg.table_schema("stackoverflow_posts")


,name,type,mode,description
0,id,INTEGER,REQUIRED,None
1,title,STRING,NULLABLE,None
2,body,STRING,NULLABLE,None
3,accepted_answer_id,INTEGER,NULLABLE,None
4,answer_count,INTEGER,NULLABLE,None
5,comment_count,INTEGER,NULLABLE,None
6,community_owned_date,TIMESTAMP,NULLABLE,None
7,creation_date,TIMESTAMP,NULLABLE,None
8,favorite_count,INTEGER,NULLABLE,None
9,last_activity_date,TIMESTAMP,NULLABLE,None


In [77]:
query = """
        SELECT 
            id, title , tags 
        FROM 
            `bigquery-public-data.stackoverflow.stackoverflow_posts`
        WHERE
            title NOT LIKE '%None%' AND 
            (tags LIKE '%|python|%' OR tags LIKE '%|c#|%' OR
            tags LIKE '%|java|%' OR tags LIKE '%|r|%' OR
            tags LIKE '%|android|%' OR tags LIKE '%|html|%' OR
            tags LIKE '%|c++|%' OR tags LIKE '%|sql|%' OR
            tags LIKE '%|c|%' OR tags LIKE '%kotlin%') AND 
            LENGTH(tags) < 20
        LIMIT
             10000;
        """

data = stack_overflow.query_to_pandas(query)

data_copy = data.copy()

data.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,id,title,tags
0,3371822,Is there anything that i can do in C but not in C++?,c++|c|oop
1,36106611,Bitmaps swallowing more memory than expected,java|android|bitmap
2,36371514,Error connect Arduino to HTML net::ERR_CONNECTION_REFUSED,php|html|arduino
3,37169462,How can I check if the device has an audio output which is not caused by my app,java|android|audio
4,18448210,Is there any library similar to conio.h?,c++|c|linux|ubuntu


**Preprocessing data**

In [78]:
data.title = data.title.str.replace('<[^<]+?>','')
data.title = data.title.str.replace('http','')
data.title = data.title.str.replace('[^\w\s]','')
data.title = data.title.str.lower()
data.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,id,title,tags
0,3371822,is there anything that i can do in c but not in c,c++|c|oop
1,36106611,bitmaps swallowing more memory than expected,java|android|bitmap
2,36371514,error connect arduino to html neterr_connection_refused,php|html|arduino
3,37169462,how can i check if the device has an audio output which is not caused by my app,java|android|audio
4,18448210,is there any library similar to conioh,c++|c|linux|ubuntu


In [79]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

data['title'] = data['title'].apply(lambda x: remove_stopwords(x))
data.head()

,id,title,tags
0,3371822,anything c c,c++|c|oop
1,36106611,bitmaps swallowing memory expected,java|android|bitmap
2,36371514,error connect arduino html neterr_connection_refused,php|html|arduino
3,37169462,check device audio output caused app,java|android|audio
4,18448210,library similar conioh,c++|c|linux|ubuntu


In [80]:
data['tags'] = data['tags'].str.split('|')
data.head()

,id,title,tags
0,3371822,anything c c,"[c++, c, oop]"
1,36106611,bitmaps swallowing memory expected,"[java, android, bitmap]"
2,36371514,error connect arduino html neterr_connection_refused,"[php, html, arduino]"
3,37169462,check device audio output caused app,"[java, android, audio]"
4,18448210,library similar conioh,"[c++, c, linux, ubuntu]"


**Preparing target variable**

In [81]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(data['tags'])

y = multilabel_binarizer.transform(data['tags'])

**Splitting Data**

In [82]:
x_train, x_val, ytrain, yval = train_test_split(data['title'],
                                                y, test_size=0.2)

**Text Preprocessing**

In [83]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=1, max_features=5000)
xtrain = tfidf_vectorizer.fit_transform(x_train)
xval = tfidf_vectorizer.transform(x_val)

**Regression Model**

In [84]:
lr = LogisticRegression()
classifier = OneVsRestClassifier(lr)
classifier.fit(xtrain, ytrain)

OneVsRestClassifier(estimator=LogisticRegression())

In [85]:
y_pred = classifier.predict(xval)
f1_score(yval, y_pred, average="micro"), accuracy_score(yval, y_pred)

(0.5458807769591426, 0.055)

**Predicting Tags**

In [86]:
pred_prob = classifier.predict_proba(xval)
t = 0.2
predp = (pred_prob >= t).astype(int)
f1_score(yval, predp, average="micro"), accuracy_score(yval, predp)

(0.6126498553121125, 0.0995)

In [87]:
def predict(m):
    m = remove_stopwords(m)
    m_vec = tfidf_vectorizer.transform([m])
    pred_prob = classifier.predict_proba(m_vec)
    t = 0.3
    predp = (pred_prob >= t).astype(int)
    #m_pred = classifier.predict(m_vec)
    return multilabel_binarizer.inverse_transform(predp)

In [88]:
for i in range(10):
    k = x_val.sample(1).index[0]
    print("Title: ", data_copy['title'][k],
          "\nPredicted tags: ", predict(x_val[k])),
    print("Actual tags: ",data['tags'][k], "\n")

Title:  Inverse Matrix in C C++ 
Predicted tags:  [('c++', 'r')]
Actual tags:  ['c++', 'c', 'matrix'] 

Title:  Android rotate pixel in bitmap but the bitmap is turn to white/mising 
Predicted tags:  [('android', 'java')]
Actual tags:  ['java', 'android', 'bitmap'] 

Title:  Make an executable at runtime 
Predicted tags:  [('html',)]
Actual tags:  ['c#', 'c++', 'c', 'executable'] 

Title:  Get value from form with button click? without submit 
Predicted tags:  [('forms', 'html', 'php')]
Actual tags:  ['php', 'html', 'ajax'] 

Title:  How to make sure no two recurring events will overlap 
Predicted tags:  [('html',)]
Actual tags:  ['mysql', 'sql', 'algorithm'] 

Title:  Libgdx SpriteBatch.SetProjectionMatrix makes spritebatch not draw 
Predicted tags:  [('android', 'java')]
Actual tags:  ['java', 'android', 'libgdx'] 

Title:  How to receive updates from server without refreshing webpage 
Predicted tags:  [('html',)]
Actual tags:  ['php', 'html', 'ajax', 'html5'] 

Title:  Wordpress ran